In [1]:
from plotnine import ggplot, geom_point, aes, stat_smooth, facet_wrap
from plotnine.data import mtcars
import pandas as pd
import numpy as np
import sys
sys.path.append("..")
from preprocessing.main import create_mappings
import os
import math
import networkx as nx
import matplotlib.pyplot as plt
import scipy

In [2]:
def extract_backbone(g, alpha):
    backbone_graph = nx.Graph()
    for node in g:
        k_n = len(g[node])
        if k_n > 1:
            sum_w = sum( g[node][neighbor]['weight'] for neighbor in g[node] )
            for neighbor in g[node]:
                edgeWeight = g[node][neighbor]['weight']
                pij = float(edgeWeight)/sum_w
                if (1-pij)**(k_n-1) < alpha: # equation 2
                    backbone_graph.add_edge( node,neighbor, weight = edgeWeight)
    return backbone_graph

In [3]:
def construct_adjacency_matrix():
    matrix = pd.DataFrame(np.zeros(shape=(len(ingredients_unique), len(ingredients_unique))),
                           columns=ingredients_unique, index=ingredients_unique)
    for m in mappings:
        ing_1_name = m[0].get_name()
        ing_2_name = m[1].get_name()
        shared_molecules = m[2]
        matrix.at[ing_1_name, ing_2_name] = shared_molecules
        matrix.at[ing_2_name, ing_1_name] = shared_molecules
    return matrix

In [17]:
mappings = create_mappings("../data")
ingredients_unique = list(set([ingredient[0].get_name() for ingredient in mappings]))

mappings_adjacency_matrix = construct_adjacency_matrix()
# mappings_adjacency_matrix = mappings_adjacency_matrix.where(np.triu(np.ones(mappings_adjacency_matrix.shape)).astype(np.bool))

In [18]:
mappings_adjacency_matrix

,green zucchini,cucumber,jalapeno,sweet potato,rocket salad,beans,salmon,vinegar,cranberry,falafel,...,avocado,onion,cheddar cheese,artichoke,tomato,capsicum,beef,beetroot,lentils,rice
green zucchini,0.0,105.0,0.0,96.0,93.0,101.0,1.0,21.0,103.0,2.0,...,102.0,96.0,22.0,95.0,103.0,107.0,13.0,105.0,101.0,98.0
cucumber,105.0,0.0,0.0,98.0,99.0,110.0,1.0,22.0,107.0,2.0,...,108.0,101.0,30.0,97.0,115.0,113.0,17.0,109.0,102.0,108.0
jalapeno,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
sweet potato,96.0,98.0,0.0,0.0,94.0,99.0,1.0,19.0,98.0,2.0,...,98.0,96.0,23.0,96.0,102.0,99.0,13.0,96.0,95.0,100.0
rocket salad,93.0,99.0,0.0,94.0,0.0,101.0,1.0,15.0,96.0,2.0,...,96.0,95.0,27.0,95.0,102.0,98.0,12.0,93.0,93.0,103.0
beans,101.0,110.0,1.0,99.0,101.0,0.0,2.0,26.0,118.0,2.0,...,112.0,106.0,57.0,95.0,142.0,121.0,23.0,105.0,100.0,132.0
salmon,1.0,1.0,0.0,1.0,1.0,2.0,0.0,1.0,3.0,1.0,...,2.0,0.0,1.0,1.0,4.0,1.0,2.0,1.0,1.0,3.0
vinegar,21.0,22.0,0.0,19.0,15.0,26.0,1.0,0.0,28.0,2.0,...,22.0,21.0,21.0,17.0,35.0,26.0,13.0,21.0,20.0,25.0
cranberry,103.0,107.0,0.0,98.0,96.0,118.0,3.0,28.0,0.0,2.0,...,112.0,103.0,47.0,95.0,135.0,120.0,27.0,103.0,102.0,125.0
falafel,2.0,2.0,0.0,2.0,2.0,2.0,1.0,2.0,2.0,0.0,...,2.0,1.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [19]:
graph = nx.from_pandas_adjacency(mappings_adjacency_matrix)
print(nx.info(graph))

Name: 
Type: Graph
Number of nodes: 50
Number of edges: 1158
Average degree:  46.3200


In [20]:
spinach = graph["spinach"]

In [21]:
sorted(spinach.items(), key=lambda edge: edge[1]['weight'], reverse=True)

[('tomato', {'weight': 114.0}),
 ('peas', {'weight': 109.0}),
 ('rice', {'weight': 109.0}),
 ('carrot', {'weight': 108.0}),
 ('potato', {'weight': 107.0}),
 ('asparagus', {'weight': 107.0}),
 ('beans', {'weight': 106.0}),
 ('capsicum', {'weight': 106.0}),
 ('mushroom', {'weight': 105.0}),
 ('cucumber', {'weight': 103.0}),
 ('olive', {'weight': 103.0}),
 ('coriander', {'weight': 103.0}),
 ('avocado', {'weight': 102.0}),
 ('flaxseed', {'weight': 101.0}),
 ('broccoli', {'weight': 101.0}),
 ('cranberry', {'weight': 100.0}),
 ('pumpkin', {'weight': 100.0}),
 ('beetroot', {'weight': 99.0}),
 ('green zucchini', {'weight': 98.0}),
 ('sweet potato', {'weight': 98.0}),
 ('rocket salad', {'weight': 98.0}),
 ('almond', {'weight': 98.0}),
 ('cashew nut', {'weight': 97.0}),
 ('artichoke', {'weight': 97.0}),
 ('onion', {'weight': 96.0}),
 ('chickpea', {'weight': 95.0}),
 ('kale', {'weight': 95.0}),
 ('pasta', {'weight': 95.0}),
 ('lentils', {'weight': 95.0}),
 ('quinoa', {'weight': 94.0}),
 ('capers'